In [1]:
import re
import numpy
import pandas

In [2]:
with open('./metabolic_network_v2.txt', 'r') as infile:
    data = pandas.read_csv(infile, delimiter = '\t', header = None)
data

,0,1,2,3
0,FUCP-MONOMER,TRANS-RXN-20-alpha,"PER-alpha-L-fucopyranose, PER-PROTON","alpha-L-fucopyranose, PROTON"
1,FUCP-MONOMER,TRANS-RXN-20-beta,"PER-beta-L-fucopyranose, PER-PROTON","beta-L-fucopyranose, PROTON"
2,FUCP-MONOMER,RXN0-7221-alpha-af,"PER-alpha-D-arabinofuranose, PER-PROTON","alpha-D-arabinofuranose, PROTON"
3,FUCP-MONOMER,RXN0-7221-beta-af,"PER-beta-D-arabinofuranose, PER-PROTON","beta-D-arabinofuranose, PROTON"
4,FUCP-MONOMER,RXN0-7221-alpha-ap,"PER-alpha-D-arabinopyranose, PER-PROTON","alpha-D-arabinopyranose, PROTON"
5,FUCP-MONOMER,RXN0-7221-beta-ap,"PER-beta-D-arabinopyranose, PER-PROTON","beta-D-arabinopyranose, PROTON"
6,FUCP-MONOMER,RXN0-7222-alpha,"PER-alpha-L-galactopyranose, PER-PROTON","alpha-L-galactopyranose, PROTON"
7,FUCP-MONOMER,RXN0-7222-alpha,"PER-alpha-L-galactopyranose, PER-PROTON","beta-L-galactopyranose, PROTON"
8,CPLX0-7645,RXN0-5298,alpha-L-fucopyranose,beta-L-fucopyranose
9,CPLX0-7631,FUCISOM-RXN,alpha-L-fucopyranose,L-FUCULOSE


In [3]:
for rxn in data.values:
    if 'CPLX' in rxn[0]:
        enzyme = 'cplx(name = \'{:s}\')'.format(rxn[0].replace('-', '_'))
    else:
        enzyme = 'prot(name = \'{:s}\')'.format(rxn[0].replace('-', '_'))
        
    name = rxn[1].replace('-', '_')
    substrates = rxn[2].replace('-', '_').split(', ')
    products = rxn[3].replace('-', '_').split(', ')

    LHS = []
    RHS = []
    
    for subs in substrates:
        if subs[0].isdigit():
            subs = '_' + subs
        
        if 'PER' in subs:
            LHS.append('met(name = \'{:s}\', loc = \'per\')'.format(subs.replace('PER_', '')))
        else:
            LHS.append('met(name = \'{:s}\', loc = \'cyt\')'.format(subs))

    for prod in products:
        if prod[0].isdigit():
            prod = '_' + prod
        
        if 'PER' in prod: # inverse transport reaction
            RHS.append('met(name = \'{:s}\', loc = \'per\')'.format(prod.replace('PER_', '')))
        else:
            RHS.append('met(name = \'{:s}\', loc = \'cyt\')'.format(prod))
            
    LHS = ' + '.join(LHS)
    RHS = ' + '.join(RHS)
        
    if rxn[0] == 'spontaneous':
        Rule = 'Rule(\'{:s}\', {:s} |\n'\
               '    {:s}, \n' \
               '    Parameter(\'fwd_{:s}\', 1), \n' \
               '    Parameter(\'rvs_{:s}\', 1))' \
               .format(name, LHS, RHS, name, name)
    
    else: # need an enzyme
        Rule = 'Rule(\'{:s}\', {:s} + {:s} | \n'\
               '    {:s} + {:s}, \n' \
               '    Parameter(\'fwd_{:s}\', 1), \n' \
               '    Parameter(\'rvs_{:s}\', 0))' \
               .format(name, enzyme, LHS, enzyme, RHS, name, name)
        
    print(Rule)
    print()

Rule('TRANS_RXN_20_alpha', prot(name = 'FUCP_MONOMER') + met(name = 'alpha_L_fucopyranose', loc = 'per') + met(name = 'PROTON', loc = 'per') | 
    prot(name = 'FUCP_MONOMER') + met(name = 'alpha_L_fucopyranose', loc = 'cyt') + met(name = 'PROTON', loc = 'cyt'), 
    Parameter('fwd_TRANS_RXN_20_alpha', 1), 
    Parameter('rvs_TRANS_RXN_20_alpha', 0))

Rule('TRANS_RXN_20_beta', prot(name = 'FUCP_MONOMER') + met(name = 'beta_L_fucopyranose', loc = 'per') + met(name = 'PROTON', loc = 'per') | 
    prot(name = 'FUCP_MONOMER') + met(name = 'beta_L_fucopyranose', loc = 'cyt') + met(name = 'PROTON', loc = 'cyt'), 
    Parameter('fwd_TRANS_RXN_20_beta', 1), 
    Parameter('rvs_TRANS_RXN_20_beta', 0))

Rule('RXN0_7221_alpha_af', prot(name = 'FUCP_MONOMER') + met(name = 'alpha_D_arabinofuranose', loc = 'per') + met(name = 'PROTON', loc = 'per') | 
    prot(name = 'FUCP_MONOMER') + met(name = 'alpha_D_arabinofuranose', loc = 'cyt') + met(name = 'PROTON', loc = 'cyt'), 
    Parameter('fwd_RXN0_722